In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2

DATADIR = "../input/trainingset/plant-detection-data"
VALDATA = "../input/validation-dataset/Validation_data"
CATEGORIES = ["Poison Oak", "Poison Ivy", "Other"]

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array)
        plt.show()
        break
    break
    
for category in CATEGORIES:
    path2 = os.path.join(VALDATA, category)
    for img in os.listdir(path2):
        img_array2 = cv2.imread(os.path.join(path2, img), cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array2)
        plt.show()
        break
    break

In [ ]:
print(img_array.shape)
print(img_array2.shape)

In [ ]:
IMG_SIZE = 40

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap = 'gray')
plt.show()

new_array2 = cv2.resize(img_array2, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array2, cmap = 'gray')
plt.show()

In [ ]:
print(new_array.shape)
print(new_array2.shape)

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                plt.imshow(img_array, cmap ='gray')
                plt.show
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
create_training_data()

val_data = []

def create_val_data():
    for category in CATEGORIES:
        path2 = os.path.join(VALDATA, category)
        class_num2 = CATEGORIES.index(category)
        for img in os.listdir(path2):
            try:
                img_array2 = cv2.imread(os.path.join(path2, img), cv2.IMREAD_GRAYSCALE)
                new_array2 = cv2.resize(img_array2, (IMG_SIZE, IMG_SIZE))
                plt.imshow(img_array2, cmap ='gray')
                plt.show
                val_data.append([new_array2, class_num2])
            except Exception as e:
                pass
create_val_data()

In [ ]:
print(len(training_data))
print(len(val_data))

In [ ]:
import random

random.shuffle(training_data)
random.shuffle(val_data)

In [ ]:
for sample in training_data[:10]:
    print(sample[1])
    
for sample in val_data[:10]:
    print(sample[1])

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

x_val = []
y_val = []
for features, label in val_data:
    x_val.append(features)
    y_val.append(label)
    
x_val = np.array(x_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = np.array(y_val)
valid_set = (x_val, y_val)
#y = np.array(y).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
#print(X)
print(valid_set)

In [ ]:
import pickle #saving dataset

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

In [ ]:
x_val[1]

In [ ]:
#creating our convolutional neural network
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

#normalize data
X = X/225.0

#building model

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", 
              optimizer="adam", 
              metrics=['accuracy'])
model.fit(X, y, batch_size=8, epochs=100, validation_data=valid_set)

model.save('PlantCNN.model')